In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape

In [30]:
# Load data
data = pd.read_parquet("Data/DSB_BDK_trainingset.parquet")

In [31]:
combinations = data[['visualiseringskode', 'station']].drop_duplicates().values
for strækning, station in combinations:
    print(data[(data['visualiseringskode'] == strækning) & (data['station'] == station)].shape)

(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)
(2557, 209)


In [32]:
def togpunktlighed_split(strækning, station, ratio=0.1):
    data_filtered = data[(data['visualiseringskode'] == strækning) & (data['station'] == station)]
    togpunktlighed = data_filtered.groupby('dato')['togpunktlighed'].mean().reset_index()
    train, test = train_test_split(togpunktlighed, test_size=ratio, shuffle=False)
    return train,test

In [33]:
# Model predictions one step into the future. Returns predictions and confidence interval
def forecast_one_step(model):
    fc, conf_int = model.predict(n_periods=1, return_conf_int=True)
    return (fc.tolist()[0], np.asarray(conf_int).tolist()[0])

In [34]:
from pmdarima import model_selection

def cv_predict(model, y, cv):
    import warnings
    warnings.filterwarnings('ignore')
    predictions = model_selection.cross_val_predict(estimator=model, y=y, cv=cv, verbose=0, averaging="median")
    return predictions

In [20]:
results_strækning_station = {}
combinations = data[['visualiseringskode', 'station']].drop_duplicates().values

for strækning, station in combinations[0:2]:
    train, test = togpunktlighed_split(strækning, station, ratio=0.1)
    y_train, y_test = train['togpunktlighed'], test['togpunktlighed']

    model = pm.auto_arima(y_train, d=1, seasonal=False, stepwise=True,
                        suppress_warnings=True, error_action="ignore", max_p=6,
                        max_order=None, trace=True)
    forecasts = []
    confidence_intervals = []

    for new_ob in y_test:
        fc, conf = forecast_one_step(model)
        forecasts.append(fc)
        confidence_intervals.append(conf)
        
        # Updates the existing model with a small number of MLE steps
        model.update(new_ob)
    results = [y_train, y_test, forecasts, confidence_intervals, model, mean_squared_error(y_test, forecasts), smape(y_test, forecasts)]
    results_strækning_station[(strækning, station)] = results
    # 80 min

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-2913.141, Time=0.76 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2484.340, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2672.171, Time=0.22 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2825.682, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-2486.340, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-2913.365, Time=0.41 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-2907.622, Time=0.34 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2915.172, Time=0.23 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-2913.336, Time=0.43 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-2786.141, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-2917.120, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-2827.661, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-2674.170, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-2915.287, Time=0.15 sec
 ARIMA(1,1,2)(0,0,0

In [44]:
from sklearn.ensemble import RandomForestRegressor
random_state = 42
regressor = RandomForestRegressor(n_estimators=100, random_state=random_state, oob_score=True, max_depth=5)

#X_train = data[(data['visualiseringskode'] == 1) & (data['station'] == 1)].iloc[:,1:] # CHANGE

#regressor.fit(X_train, y_train)

In [46]:
from pmdarima import model_selection

smape_scores = []
for (strækning, station), results in results_strækning_station.items():
    y_train, y_test, forecasts, confidence_intervals, model, _, _ = results
    y = pd.concat([y_train, y_test])
    cv = model_selection.RollingForecastCV(initial=100, step=1, h=1)
    predictions = cv_predict((model, regressor), y, cv)
    print(1)

AttributeError: 'tuple' object has no attribute 'fit'

In [35]:
X = data.iloc[:,1:]

In [25]:
data.groupby('dato')

In [42]:
data[(data['visualiseringskode'] == 1) & (data['station'] == 1)].iloc[:,1:]

,koreplan_antal_planlagte_unik_tognr_togpkt_myldretid,koreplan_antal_planlagte_prod_kode_myldretid,koreplan_stdafv_antal_unik_prod_kode_per_time_myldretid,koreplan_antal_planlagte_unik_tognr_alle_myldretid,koreplan_stdafv_antal_unik_tognr_alle_per_time_myldretid,koreplan_antal_planlagte_unik_tognr_alle_g_meld_myldretid,koreplan_antal_planlagte_togfoelge_under_5_min_myldretid,koreplan_gns_planlagte_togfoelg_alle_min_myldretid,koreplan_stdafv_planlagte_togfoelg_alle_min_myldretid,koreplan_antal_planlagte_unik_tognr_godstog_myldretid,...,dsb_tgf_fte,dsb_tgf_raadighed,dsb_tgf_sygdom,dsb_tgf_relative_sygdom,dsb_lkf_fte,dsb_lkf_sygdom,dsb_lkf_relative_sygdom,dsb_haandvaerker,dsb_fravaer,dsb_relative_fravaer
dato,,,,,,,,,,,,,,,,,,,,,
2016-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,48.35664,0.0,0.0,0.00000,67.500031,0.000000,0.000000,482.0,4.594,0.047412
2016-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,48.35664,0.0,0.0,0.00000,67.500031,8.000000,0.118518,482.0,4.594,0.047412
2016-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,48.35664,0.0,0.0,0.00000,67.500031,17.469999,0.258815,482.0,4.594,0.047412
2016-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,48.35664,0.0,0.0,0.00000,67.500031,0.000000,0.000000,482.0,17.822,0.143220
2016-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,48.35664,19.6,1.0,0.02068,67.500031,0.000000,0.000000,482.0,23.822,0.194415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,12.0,3.0,0.0,12.0,0.0,0.0,0.0,63.27,110.9,0.0,...,0.00000,0.0,0.0,0.00000,41.000000,7.920000,0.193171,493.0,9.343,0.059891
2022-12-28,12.0,3.0,0.0,12.0,0.0,0.0,0.0,63.27,110.9,0.0,...,0.00000,0.0,0.0,0.00000,41.000000,17.190001,0.419268,493.0,11.343,0.072712
2022-12-29,12.0,3.0,0.0,12.0,0.0,0.0,0.0,63.27,110.9,0.0,...,0.00000,0.0,0.0,0.00000,41.000000,7.920000,0.193171,493.0,11.343,0.072712


In [37]:
X

,koreplan_antal_planlagte_unik_tognr_togpkt_myldretid,koreplan_antal_planlagte_prod_kode_myldretid,koreplan_stdafv_antal_unik_prod_kode_per_time_myldretid,koreplan_antal_planlagte_unik_tognr_alle_myldretid,koreplan_stdafv_antal_unik_tognr_alle_per_time_myldretid,koreplan_antal_planlagte_unik_tognr_alle_g_meld_myldretid,koreplan_antal_planlagte_togfoelge_under_5_min_myldretid,koreplan_gns_planlagte_togfoelg_alle_min_myldretid,koreplan_stdafv_planlagte_togfoelg_alle_min_myldretid,koreplan_antal_planlagte_unik_tognr_godstog_myldretid,...,dsb_tgf_fte,dsb_tgf_raadighed,dsb_tgf_sygdom,dsb_tgf_relative_sygdom,dsb_lkf_fte,dsb_lkf_sygdom,dsb_lkf_relative_sygdom,dsb_haandvaerker,dsb_fravaer,dsb_relative_fravaer
dato,,,,,,,,,,,,,,,,,,,,,
2016-01-01,5.0,2.0,0.41,5.0,0.41,0.0,0.0,150.00,180.00,0.0,...,48.356640,0.000000,0.0,0.000000,67.500031,0.000000,0.000000,482.0,4.594,0.047412
2016-01-01,8.0,2.0,0.82,8.0,0.82,0.0,0.0,85.57,129.41,0.0,...,48.356640,0.000000,0.0,0.000000,67.500031,0.000000,0.000000,482.0,4.594,0.047412
2016-01-01,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,48.356640,0.000000,0.0,0.000000,67.500031,0.000000,0.000000,482.0,4.594,0.047412
2016-01-01,13.0,3.0,1.33,13.0,1.33,0.0,0.0,50.00,107.40,0.0,...,163.808955,117.900002,4.0,0.024419,186.132726,20.680000,0.111104,482.0,4.594,0.047412
2016-01-01,16.0,4.0,1.51,16.0,1.51,0.0,3.0,46.00,94.66,0.0,...,163.808955,117.900002,4.0,0.024419,186.132726,20.680000,0.111104,482.0,4.594,0.047412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.000000,0.000000,0.0,0.000000,29.000000,22.520000,0.776552,493.0,6.379,0.040891
2022-12-31,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,100.505023,34.000000,4.0,0.039799,102.000000,23.799999,0.233333,493.0,6.379,0.040891
2022-12-31,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,148.977671,34.600000,1.0,0.006712,97.000000,24.270000,0.250206,493.0,6.379,0.040891


In [94]:
import numpy as np

def custom_cross_val_predict(estimator, y, X=None, cv=None, verbose=0, averaging="mean", return_raw_predictions=False):
    """Generate cross-validated estimates for each input data point
    
    Parameters
    ----------
    estimator : tuple
        A tuple containing two estimators. The first estimator should be the ARIMA model
        and the second one should be the Random Forest model.

    y : array-like or iterable, shape=(n_samples,)
        The time-series array.

    X : array-like, shape=[n_obs, n_vars], optional (default=None)
        An optional 2-d array of exogenous variables.

    cv : BaseTSCrossValidator or None, optional (default=None)
        An instance of cross-validation. If None, will use a RollingForecastCV.
        Note that for cross-validation predictions, the CV step cannot exceed
        the CV horizon, or there will be a gap between fold predictions.

    verbose : integer, optional
        The verbosity level.

    averaging : str or callable, one of ["median", "mean"] (default="mean")
        Unlike normal CV, time series CV might have different folds (windows)
        forecasting the same time step. After all forecast windows are made,
        we build a matrix of y x n_folds, populating each fold's forecasts like
        so::

            nan nan nan  # training samples
            nan nan nan
            nan nan nan
            nan nan nan
              1 nan nan  # test samples
              4   3 nan
              3 2.5 3.5
            nan   6   5
            nan nan   4

        We then average each time step's forecasts to end up with our final
        prediction results.

    return_raw_predictions : bool (default=False)
        If True, raw predictions are returned instead of averaged ones.
        This results in a y x h matrix. For example, if h=3, and step=1 then:

            nan nan nan # training samples
            nan nan nan
            nan nan nan
            nan nan nan
            1   4   2   # test samples
            2   5   7
            8   9   1
            nan nan nan
            nan nan nan

        First column contains all one-step-ahead-predictions, second column all
        two-step-ahead-predictions etc. Further metrics can then be calculated
        as desired.

    Returns
    -------
    predictions : array-like, shape=(n_samples,)
        The predicted values.

    """

    def indexable(*iterables):
        """Internal utility to handle input types"""
        results = []
        for iterable in iterables:
            if not hasattr(iterable, "__iter__"):
                raise ValueError("Input {!r} is not indexable".format(iterable))
            results.append(iterable)
        return results

    def check_cv(cv):
        """Internal utility to check cv"""
        if cv is None:
            from pmdarima.model_selection import RollingForecastCV
            cv = RollingForecastCV(initial=2554, step=1, h=1)
        return cv

    def check_endog(y, copy=True, preserve_series=False):
        """Internal utility to check endogenous variable"""
        from pmdarima.utils import check_endog
        return check_endog(y, copy=copy, preserve_series=preserve_series)

    def _check_averaging(averaging):
        """Internal utility to check averaging"""
        if averaging == "mean":
            return np.nanmean
        elif averaging == "median":
            return np.nanmedian
        elif callable(averaging):
            return averaging
        else:
            raise ValueError("Unknown averaging method: {}".format(averaging))

    def _fit_and_predict(fold, estimator_tuple, y, X, train, test, verbose=0):
        """Internal utility to fit and predict"""
        arima_model, rf_model = estimator_tuple
        # Fit ARIMA model
        arima_model.fit(y[train]) # X=X.iloc[train, :]
        # Predict with ARIMA model
        train_predictions = model.predict_in_sample()
        arima_pred = arima_model.predict(n_periods=len(test))

        # Calculate residuals
        arima_residuals_train = train_predictions - y[train]
        arima_residuals_test = np.array(arima_pred - y[test])
        # Fit Random Forest model on ARIMA residuals
        rf_model.fit(X.iloc[train, :], arima_residuals_train)

        # Predict with Random Forest model
        rf_pred = rf_model.predict(X=X.iloc[test, :])

        # Overall prediction residuals = pred - true <=> true = pred - residuals
        overall_pred = arima_pred - rf_pred

        return overall_pred, test #arima_residuals_test

    y, X = indexable(y, X)
    y = check_endog(y, copy=False, preserve_series=True)
    cv = check_cv(cv)
    avgfunc = _check_averaging(averaging)

    if cv.step > cv.horizon:
        raise ValueError("CV step cannot be > CV horizon, or there will be a gap in predictions between folds")

    prediction_blocks = [
        _fit_and_predict(fold,
                         estimator,
                         y,
                         X,
                         train=train,
                         test=test,
                         verbose=verbose,)  # TODO: fit params?
        for fold, (train, test) in enumerate(cv.split(y, X))]

    pred_matrix = np.ones((y.shape[0], len(prediction_blocks))) * np.nan
    for i, (pred_block, test_indices) in enumerate(prediction_blocks):
        print(test_indices)
        pred_matrix[test_indices, i] = pred_block

    if return_raw_predictions:
        predictions = np.ones((y.shape[0], cv.horizon)) * np.nan
        for pred_block, test_indices in prediction_blocks:
            predictions[test_indices[0]] = pred_block
        return predictions

    test_mask = ~(np.isnan(pred_matrix).all(axis=1))
    predictions = pred_matrix[test_mask]
    return avgfunc(predictions, axis=1)


In [95]:
y = data[(data['visualiseringskode'] == 1) & (data['station'] == 1)]['togpunktlighed']
X = data[(data['visualiseringskode'] == 1) & (data['station'] == 1)].iloc[:,1:]
custom_cross_val_predict((model,regressor), y, X, cv=None, verbose=0, averaging="mean", return_raw_predictions=False)
# output = prediction per fold.

[2554]
[2555]
[2556]


array([1.00433547, 0.99587514, 0.94213564])

In [91]:
y[-3:]

dato
2022-12-29    1.000
2022-12-30    0.906
2022-12-31    1.000
Name: togpunktlighed, dtype: float64

In [58]:
data[(data['visualiseringskode'] == 1) & (data['station'] == 1)]['togpunktlighed']

dato
2016-01-01    0.000
2016-01-02    0.000
2016-01-03    0.000
2016-01-04    0.000
2016-01-05    0.000
              ...  
2022-12-27    0.906
2022-12-28    1.000
2022-12-29    1.000
2022-12-30    0.906
2022-12-31    1.000
Name: togpunktlighed, Length: 2557, dtype: float64